<a href="https://colab.research.google.com/github/minhaj3/gpt-finetuning-colab/blob/master/GPT_117M_FB_MSG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import json
import random
import re

In [2]:
!git clone -b small https://github.com/Tenoke/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 244, done.
remote: Total 244 (delta 0), reused 0 (delta 0), pack-reused 244
Receiving objects: 100% (244/244), 4.39 MiB | 14.68 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [3]:
cd gpt-2

/content/gpt-2


In [42]:
!ls

checkpoint	   fb-cleaned-medha.txt  LICENSE	   samples
Dockerfile.cpu	   fb-cleaned.txt	 messages	   src
Dockerfile.gpu	   fb-cleaned.txt.npz	 models		   train.py
download_model.sh  fb-json.zip		 README.md
encode.py	   gpt-2-samples	 requirements.txt


In [14]:
!cat download_model.sh

#!/bin/sh

if [ "$#" -ne 1 ]; then
    echo "You must enter the model name as a parameter, e.g.: sh download_model.sh 117M"
    exit 1
fi

model=$1

mkdir -p models/$model

# TODO: gsutil rsync -r gs://gpt-2/models/ models/
for filename in checkpoint encoder.json hparams.json model.ckpt.data-00000-of-00001 model.ckpt.index model.ckpt.meta vocab.bpe; do
  fetch=$model/$filename
  echo "Fetching $fetch"
  curl --output models/$fetch https://storage.googleapis.com/gpt-2/models/$fetch
done


In [6]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 3.8MB/s 
     |████████████████████████████████| 604kB 10.1MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=8244b71ca2f20a9bec89432042005fc51837616254c991a271005738c8726b83
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533170 sha256=40850c3bdc2f363383a4865b6d51c068138032c98448db233a85b8c0e5b4acc8
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex


In [8]:
!sh download_model.sh 117M

Fetching 117M/checkpoint
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    77  100    77    0     0    726      0 --:--:-- --:--:-- --:--:--   733
Fetching 117M/encoder.json
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1017k  100 1017k    0     0  8208k      0 --:--:-- --:--:-- --:--:-- 8208k
Fetching 117M/hparams.json
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    90  100    90    0     0   1267      0 --:--:-- --:--:-- --:--:--  1267
Fetching 117M/model.ckpt.data-00000-of-00001
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  47

In [12]:
#!curl 'https://bigzipfiles.facebook.com/p/dl/download/file.php?r=100000045203418&t=100000045203418&j=11&i=2423522277659255&ext=1552347802&hash=AaDXnDMervVmbTPV' -H 'authority: bigzipfiles.facebook.com' -H 'upgrade-insecure-requests: 1' -H 'dnt: 1' -H 'user-agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36' -H 'accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8' -H 'referer: https://www.facebook.com/' -H 'accept-encoding: gzip, deflate, br' -H 'accept-language: en-GB,en;q=0.9,bg-BG;q=0.8,bg;q=0.7,en-US;q=0.6' -H 'cookie: datr=qi5tXEI5YtkypxAdISqJGNZd; sb=qi5tXJPGdhJhbt5N04dUGcUv; c_user=100000045203418; xs=29%3AgDb81L_oJkC4Kg%3A2%3A1550659246%3A4402%3A5224; pl=n; dpr=0.8999999761581421; spin=r.1000481805_b.trunk_t.1552342837_s.1_v.2_; fr=1BXrWEDVIeChp7NLC.AWXvyWCWdyCZAdTM2bSw40pj8-E.BcbS6q.Hm.AAA.0.0.BchuAL.AWWf6I2i; presence=EDvF3EtimeF1552346587EuserFA21BB45203418A2EstateFDsb2F1552344421952EatF1552345159684Et3F_5bDiFA2user_3a1B00305017170A2EoF1EfF1CAcDiFA2user_3a1B01308116906A2EoF2EfF3CAcDiFA2user_3a1B08124067081A2EoF3EfF2C_5dEutc3F1552345156737G552346587474CEchFDp_5f1BB45203418F1CC; act=1552346589369%2F2; wd=901x697; pnl_data2=eyJhIjoib25hZnRlcmxvYWQiLCJjIjoiWERZSVJlcXVlc3RDb250cm9sbGVyIiwiYiI6ZmFsc2UsImQiOiIvcC9kbC9kb3dubG9hZC9maWxlLnBocCIsImUiOltdfQ%3D%3D' --compressed --output fb-json.zip
!curl 'https://bigzipfiles.facebook.com/p/dl/download/file.php?r=100002213815538&t=100002213815538&j=11&i=2511412695609166&f=2511419428941826&ext=1571498646&hash=AaBdbiByc0GNk936' -H 'User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8' -H 'Accept-Language: en-US,en;q=0.5' --compressed -H 'Referer: https://www.facebook.com/' -H 'DNT: 1' -H 'Connection: keep-alive' -H 'Cookie: fr=1kqjDg4tKBuaeNIlM.AWXtXHVfhF0iX8j3iL5TzspBuT8.BdqvWe.ka.AAA.0.0.Bdqx_S.AWUV6qw2; wd=1525x429; datr=nvWqXTbYKGPWUb-Qml8vXa28; sb=tfWqXYj5C6mdrim5_8_3PwdR; locale=en_GB; c_user=100002213815538; xs=28%3AZjeg_deLGi5x7Q%3A2%3A1571485372%3A12391%3A4201; spin=r.1001317877_b.trunk_t.1571485374_s.1_v.2_; dpr=0.8955223880597015; act=1571496210395%2F4; presence=EDvF3EtimeF1571495938EuserFA21B02213815538A2EstateFDutF1571495938462CEchFDp_5f1B02213815538F1CC; pnl_data2=eyJhIjoib25hZnRlcmxvYWQiLCJjIjoiWERZSVJlcXVlc3RDb250cm9sbGVyIiwiYiI6ZmFsc2UsImQiOiIvcC9kbC9kb3dubG9hZC9maWxlLnBocCIsImUiOltdfQ%3D%3D' -H 'Upgrade-Insecure-Requests: 1' -H 'TE: Trailers'  --compressed --output fb-json.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 39.7M  100 39.7M    0     0  22.0M      0  0:00:01  0:00:01 --:--:-- 22.0M


In [0]:
!unzip fb-json.zip

In [0]:
!ls -l -h 

In [21]:
files = []
for p, d, f in os.walk('messages/inbox'):
    for file in f:
        if file.endswith('message_1.json'):
            files.append(f'{p}/{file}')

len(files)

189

In [0]:
def fix_encoding(s):
  return re.sub('[\xc2-\xf4][\x80-\xbf]+',lambda m: m.group(0).encode('latin1').decode('utf8'),s)

def find_cyrilic(s):
    return len(re.findall('(?i)[А-ЯЁ]', s)) > 0

def test_mostly_cyrilic(messages):
  i = 0
  check_n = min(250, len(messages))
  for msg in random.sample(messages, check_n):
    try:
      i +=find_cyrilic(fix_encoding(msg['content'])) or find_cyrilic(fix_encoding(msg['sender_name']))
    except KeyError:
      check_n -=1
  return i > check_n/5

In [26]:
!python --version

Python 3.6.8


In [0]:

text_corpus = ''
banned_names = ()
for file in files:
  with open(file, 'r') as f:
      try:
        msgs = json.load(f)['messages']
        msgs.reverse()
      except:
        pass
      else:
        if not test_mostly_cyrilic(msgs) and not any(bn in file for bn in banned_names):
          for msg in msgs:
            try:
              content = fix_encoding(msg['content'])
              to_add  = f"({msg['timestamp_ms']}) {msg['sender_name']}: {content}\n"
              if not find_cyrilic(to_add):
                text_corpus += to_add
            except KeyError:
              pass
          print(file)

          text_corpus += '\n\n'

 

In [0]:
with open('fb-cleaned.txt', 'w') as f:
  f.write(text_corpus)

In [25]:
!du -h fb-cleaned.txt

2.0M	fb-cleaned.txt


In [27]:
!cat encode.py

#!/usr/bin/env python3
# Usage:
#  PYTHONPATH=src ./encode.py <file|directory|glob> /path/to/output.npz
#  PYTHONPATH=src ./train --dataset /path/to/output.npz

import fire
import json
import os
import numpy as np
import tensorflow as tf
import random
import time
import tqdm
import glob

import encoder


def load_dataset(enc, path):
    paths = []
    if os.path.isfile(path):
        # Simple file
        paths.append(path)
    elif os.path.isdir(path):
        # Directory
        for (dirpath, _, fnames) in os.walk(path):
            for fname in fnames:
                paths.append(os.path.join(dirpath, fname))
    else:
        # Assume glob
        paths = glob.glob(path)

    token_chunks = []
    for path in tqdm.tqdm(paths):
        with open(path, 'r') as fp:
            raw_text = fp.read()
        tokens = np.stack(enc.encode(raw_text))
        token_chunks.append(tokens)
    return token_chunks


def encode_main(in_text, out_npz, model_name='117M'):
    enc = encoder.get_enc

In [46]:
!PYTHONPATH=src ./encode.py --in-text fb-cleaned-junaid.txt --out-npz fb-cleaned.txt.npz

Reading files
100% 1/1 [00:03<00:00,  3.81s/it]
Writing fb-cleaned.txt.npz


In [0]:
!PYTHONPATH=src ./train.py --dataset fb-cleaned.txt.npz --sample_every=250 --learning_rate=0.0001 --stop_after=251


In [0]:
!PYTHONPATH=src ./train.py --dataset fb-cleaned.txt.npz --sample_every=250 --learning_rate=0.001 --stop_after=751


In [0]:
!PYTHONPATH=src ./train.py --dataset fb-cleaned.txt.npz --sample_every=250 --learning_rate=0.0001 --beta=0.95 --stop_after=1251

In [0]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/117M/

In [0]:
!python3 src/generate_unconditional_samples.py --top_k 40 --temperature 0.9

In [0]:
!python3 src/interactive_conditional_samples.py --top_k 40 --temperature 0.9

In [0]:
def create_specific_file(person, files=files):
  text_corpus = ''
  for file in files:
    if person in file:
      print(file)
      with open(file, 'r') as f:
          try:
            msgs = json.load(f)['messages']
            msgs.reverse()
          except:
            pass
          else:
            for msg in msgs:
              try:
                content = fix_encoding(msg['content'])
                to_add  = f"({msg['timestamp_ms']}) {msg['sender_name']}: {content}\n"
                if not find_cyrilic(to_add):
                  text_corpus += to_add
              except KeyError:
                pass

              text_corpus += '\n\n'
      with open(f'fb-cleaned-{person}.txt', 'w') as f:
        f.write(text_corpus)
        return


In [0]:
create_specific_file('junaid')

In [44]:
!ls

checkpoint	   fb-cleaned-medha.txt  LICENSE	   samples
Dockerfile.cpu	   fb-cleaned.txt	 messages	   src
Dockerfile.gpu	   fb-cleaned.txt.npz	 models		   train.py
download_model.sh  fb-json.zip		 README.md
encode.py	   gpt-2-samples	 requirements.txt
